# Исследование данных телекоммуникационной компании
Цель исследования - определение текущего уровня потребительской лояльности или NPS.

Источник данных - статистические данные телекоммуникационной компании.

Задачи исследования - проведение предобработки данных, построение распределения признаков и определение взаимосвязей, построение и оценка метрик моделей оттока клиентов, создание кластеризации клиентов, формулировка выводов и подготовка базовых рекомендаций по работе с клиентами сети фитнес-центров "Культурист-датасаентист".

Этапы исследования:
1. подключение к базе,
2. выгрузка данных, 
3. создание дашбордов в Tableau,
4. ответы на вопросы исследования и поиск интересных особенностей.

In [3]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [4]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [5]:
query = """ 
    SELECT u.user_id,
           u.lt_day,
           CASE 
               WHEN ls.bucket_max <= 12 THEN '1'
               ElSE '0'
               END AS is_new,
           u.age,
           u.gender_segment,
           u.os_name,
           u.cpe_type_name,
           l.country,
           l.city,
           ag.title AS age_segment,
           ts.title AS traffic_segment,
           ls.title AS lifetime_segment,
           u.nps_score,
           CASE 
               WHEN u.nps_score >= 9 THEN 'promoters'
               WHEN u.nps_score = '7' or  u.nps_score = '8' THEN 'passives'
               WHEN u.nps_score <= '6' THEN 'detractors'
               END AS nps_group
                      
    FROM user as u
    LEFT OUTER JOIN location as l ON u.location_id=l.location_id
    LEFT OUTER JOIN age_segment as ag ON u.age_gr_id=ag.age_gr_id
    LEFT OUTER JOIN lifetime_segment as ls ON u.lt_gr_id=ls.lt_gr_id
    LEFT OUTER JOIN traffic_segment as ts ON u.tr_gr_id=ts.tr_gr_id
    ; 
    """ 

In [6]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,0,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,0,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters


In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Ссылка на дашборды, презентацию и выводы: https://public.tableau.com/views/2_16842106142380/sheet2?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link